In [1]:
import QuantLib as ql
import openpyxl as xl

In [2]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [3]:
PstBook = xl.load_workbook('Pst_CPBA.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
CCY = MktSheet.cell(5, 1).value
Type = MktSheet.cell(5, 2).value
MTMDate = MktSheet.cell(1, 3).value
settings = ql.Settings.instance()
evDate = ql.Date(MTMDate.day, MTMDate.month, MTMDate.year)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
DC365 = ql.Actual365Fixed()
BDC = ql.Unadjusted

In [4]:
CPSP = MktSheet.cell(13, 8).value
# NCD3M = MktSheet.cell(14, 8).value
CP6M = MktSheet.cell(15, 8).value
CP12M = MktSheet.cell(16, 8).value
DateSP = evDate
# Date3M = Cal.advance(evDate, 3, ql.Months, BDC, False)
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date12M = Cal.advance(evDate, 12, ql.Months, BDC, False)
ListDate = list([DateSP, Date6M, Date12M])
ListRate = list([CPSP, CP6M, CP12M])
VecDate = ql.DateVector(ListDate)
VecRate = ql.DoubleVector(ListRate)
TSZero = ql.ZeroCurve(VecDate, VecRate, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)

In [5]:
CPBASheet = PstBook['CPBA']
Rows = CPBASheet.rows
TRows = tuple(Rows) # 6
start_row = 2
end_row = len(TRows)
start_col = 0 # 'A' column index
end_col = 5 # 'E' column index
ListCPBA = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in CPBASheet[i][start_col:end_col + 1]]
    ListCPBA.append(row)
    print(row) # list of cell values of this row

['CPBA001', 'Mega', datetime.datetime(2021, 6, 18, 0, 0), datetime.datetime(2021, 9, 17, 0, 0), 91, 200000000]
['CPBA002', 'Mega', datetime.datetime(2021, 6, 23, 0, 0), datetime.datetime(2021, 7, 21, 0, 0), 28, 700000000]
['CPBA003', 'Mega', datetime.datetime(2021, 6, 25, 0, 0), datetime.datetime(2021, 7, 23, 0, 0), 28, 200000000]
['CPBA004', 'CTBC', datetime.datetime(2021, 6, 29, 0, 0), datetime.datetime(2021, 7, 27, 0, 0), 28, 1000000000]
['CPBA005', 'CTBC', datetime.datetime(2021, 6, 30, 0, 0), datetime.datetime(2021, 7, 28, 0, 0), 28, 200000000]


In [6]:
CPBABook = list()
for i in range(len(ListCPBA)):
    row = ListCPBA[i]
    record = list()
    record.append(row[0])
    record.append(row[1])
    record.append(DatetimeToDate(row[2]))
    record.append(DatetimeToDate(row[3]))
    record.append(row[4])
    record.append(row[5])
    CPBABook.append(record)

In [7]:
CPBABook

[['CPBA001', 'Mega', Date(18,6,2021), Date(17,9,2021), 91, 200000000],
 ['CPBA002', 'Mega', Date(23,6,2021), Date(21,7,2021), 28, 700000000],
 ['CPBA003', 'Mega', Date(25,6,2021), Date(23,7,2021), 28, 200000000],
 ['CPBA004', 'CTBC', Date(29,6,2021), Date(27,7,2021), 28, 1000000000],
 ['CPBA005', 'CTBC', Date(30,6,2021), Date(28,7,2021), 28, 200000000]]

In [8]:
h_TS = ql.YieldTermStructureHandle(TSZero)
CPEngine = ql.DiscountingBondEngine(h_TS)

In [9]:
ListZero = ListRate
VecDate = ql.DateVector(ListDate)

In [10]:
ListZero

[0.003872683592214857, 0.005679705459626832, 0.008366007168002142]

In [11]:
NoCurve = len(ListZero)
print(NoCurve)
ListRW = list([0.05, 0.05, 0.05, 0.05, 0.05])

3


In [12]:
ListListZero = list()
ListListZero.append(ListZero)
# ZoneTuple = (2, 3)
for i in range(1, NoCurve):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoCurve):
        if(i==j):
            NewList.append(ListZero[j] + 0.0001)
        else:
            NewList.append(ListZero[j])
    ListListZero.append(NewList)

In [13]:
ListListZero

[[0.003872683592214857, 0.005679705459626832, 0.008366007168002142],
 [0.003872683592214857, 0.005779705459626833, 0.008366007168002142],
 [0.003872683592214857, 0.005679705459626832, 0.008466007168002141]]

In [14]:
ListVecZero = list()
ListTSZero = list()
for i in range(NoCurve):
    AVecZero = ql.DoubleVector(ListListZero[i])
    ListVecZero.append(AVecZero)
    ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
    ListTSZero.append(ATSZero)

In [15]:
ListTSZero

[<QuantLib.ZeroCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedZeroCurve< Linear > > *' at 0x000001FEE0137750> >,
 <QuantLib.ZeroCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedZeroCurve< Linear > > *' at 0x000001FEE01378A0> >,
 <QuantLib.ZeroCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedZeroCurve< Linear > > *' at 0x000001FEE01379C0> >]

In [16]:
CurvUpListListZero = list()
CurvUpListListZero.append(ListZero)
for i in range(1, NoCurve):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoCurve):
        if(i==j):
            NewList.append(ListZero[j] + 0.05)
        else:
            NewList.append(ListZero[j])
    CurvUpListListZero.append(NewList)

In [17]:
CurvUpListListZero

[[0.003872683592214857, 0.005679705459626832, 0.008366007168002142],
 [0.003872683592214857, 0.010679705459626833, 0.008366007168002142],
 [0.003872683592214857, 0.005679705459626832, 0.008366007168002142]]

In [18]:
CurvDnListListZero = list()
CurvDnListListZero.append(ListZero)
for i in range(1, NoCurve):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoCurve):
        if(i==j):
            NewList.append(ListZero[j] - 0.05)
        else:
            NewList.append(ListZero[j])
    CurvDnListListZero.append(NewList)

In [19]:
CurvDnListListZero

[[0.003872683592214857, 0.005679705459626832, 0.008366007168002142],
 [0.003872683592214857, 0.0006797054596268323, 0.008366007168002142],
 [0.003872683592214857, 0.005679705459626832, 0.0033660071680021416]]

In [20]:
ListListZero[0]

[0.003872683592214857, 0.005679705459626832, 0.008366007168002142]

In [21]:
ListListZero[1]

[0.003872683592214857, 0.005779705459626833, 0.008366007168002142]

In [22]:
ListListZero[2]

[0.003872683592214857, 0.005679705459626832, 0.008466007168002141]

In [23]:
SenSheet = PstBook['CSR']
for i in range(0, len(CPBABook)):
    ListValue = list()
    record = CPBABook[i]
    startDate = record[2]
    maturityDate = record[3]
    faceAmt = record[5]
    CP = ql.ZeroCouponBond(0, Cal, faceAmt, maturityDate, BDC, faceAmt)
    
    h_TS = ql.YieldTermStructureHandle(ListTSZero[0])
    CPEngine = ql.DiscountingBondEngine(h_TS)
    CP.setPricingEngine(CPEngine)
    Value = CP.dirtyPrice()
    
    ListValue.append(Value)
    for j in range(1, NoCurve):
        h_TS = ql.YieldTermStructureHandle(ListTSZero[j])
        CPEngine = ql.DiscountingBondEngine(h_TS)
        CP.setPricingEngine(CPEngine)
        Value = CP.dirtyPrice()
        ListValue.append(Value)
        SenSheet.cell(i+5, 1).value = record[0]
        SenSheet.cell(i+5, 2).value = record[1]
        SenSheet.cell(i+5, 3).value = ListValue[0]        
    
    ListSensitivity = list()
    for j in range(1, len(ListValue)):
        ListSensitivity.append((ListValue[j] - ListValue[0])*10000)
    
    ListWS = list()
    for j in range(0, len(ListSensitivity)):
        ListWS.append(ListRW[j] * ListSensitivity[j])    
    
    for j in range(0, len(ListSensitivity)):    
        SenSheet.cell(i+5, j+5).value = ListWS[j]

    ListVecZero = list()
    CurvUpListTSZero = list()
    for j in range(NoCurve):
        AVecZero = ql.DoubleVector(CurvUpListListZero[j])
        ListVecZero.append(AVecZero)
        ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
        CurvUpListTSZero.append(ATSZero)
                
    ListVecZero = list()
    CurvDnListTSZero = list()
    for j in range(NoCurve):
        AVecZero = ql.DoubleVector(CurvDnListListZero[j])
        ListVecZero.append(AVecZero)
        ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
        CurvDnListTSZero.append(ATSZero)
        
    CurvUpListHTSZero = list()
    for j in range(NoCurve):
        CurvUpListHTSZero.append(ql.YieldTermStructureHandle(CurvUpListTSZero[j]))
        
    CurvDnListHTSZero = list()
    for j in range(NoCurve):
        CurvDnListHTSZero.append(ql.YieldTermStructureHandle(CurvDnListTSZero[j]))
        
    CurvUpValue = list()
    for j in range(NoCurve):
        CPEngine = ql.DiscountingBondEngine(CurvUpListHTSZero[j])
        CP.setPricingEngine(CPEngine)
        Value = CP.dirtyPrice()
        CurvUpValue.append(Value)
    
    CurvDnValue = list()
    for j in range(NoCurve):
        CPEngine = ql.DiscountingBondEngine(CurvDnListHTSZero[j])
        CP.setPricingEngine(CPEngine)
        Value = CP.dirtyPrice()
        CurvDnValue.append(Value)        
        
    CurvatureUp = list()
    ShiftUpValue = list()
    DeltaChange = list()
    for j in range(1, NoCurve):
        ShiftUpValue.append(CurvUpValue[j]-CurvUpValue[0])        
        CurvatureUp.append(-((CurvUpValue[j]-CurvUpValue[0]) - 0.05 * ListSensitivity[j-1]))
                
    for j in range(0, len(CurvatureUp)):    
        SenSheet.cell(i+5, j+12).value = CurvatureUp[j]
            
    CurvatureDn = list()
    ShiftDnValue = list()
    for j in range(1, NoCurve):
        ShiftDnValue.append(CurvDnValue[j]-CurvDnValue[0])    
        CurvatureDn.append(-((CurvDnValue[j]-CurvDnValue[0]) + 0.05 * ListSensitivity[j-1]))

    for j in range(0, len(CurvatureDn)):    
        SenSheet.cell(i+5, j+19).value = CurvatureDn[j]
        
PstBook.save('Pst_CPBA.xlsx')
PstBook.close()

In [24]:
ListValue

[199936370.87350824, 199936136.872435, 199936370.87350824]